In [1]:
import pandas as pd
coffee_path = r'C:\Users\44771\Documents\Repo\Data_projects_24\coffee_survey\coffee_data.csv'
coffee_df = pd.read_csv(coffee_path)

def reformat_mc_q(dataframe, list_of_cols: list, prefix_to_remove: str, var_name: str):
    '''
    This function will take a subset of columns from the dataset (the submission ID and data from a multiple choice question).
    It cleans this subset of data by dropping rows containing NULL values and removing the unnecessary prefixes and brackets in the columns.
    It then melts the dataset so we go from wide format to long-format.

    Parameters:
    prefix_to_remove (str) : The prefix to remove from the column names.
    var_name (str) : The name to assign to the variable column created during the melting process.
    list_of_cols (list) : A list of column names to include in the subset of the DataFrame. This should include the 'Submission ID' and the columns representing the multiple choice questions.
    dataframe (pandas dataframe) : The original DataFrame containing the data to be processed.

    Returns:
    final_df (pandas dataframe): A DataFrame with two columns- 'Submission ID' and the options each user selected in the multiple choice question.

    '''
    df = dataframe[list_of_cols].dropna()
    df.columns = df.columns.str.replace(prefix_to_remove, '', regex=False)
    df.columns = df.columns.str.replace(r'\(|\)', '', regex=True).str.strip()
    df_melt = df.melt(id_vars='Submission ID',var_name=var_name)
    final_df = df_melt[df_melt['value']==True].drop(columns='value')
    return final_df

In [2]:
why_q = [
    'Submission ID',
    'Why do you drink coffee? (It tastes good)',
    'Why do you drink coffee? (I need the caffeine)',
    'Why do you drink coffee? (I need the ritual)',
    'Why do you drink coffee? (It makes me go to the bathroom)',
    'Why do you drink coffee? (Other)'
]

why_df = reformat_mc_q(coffee_df,why_q,'Why do you drink coffee?','Why do you drink coffee?')
why_df.sort_values('Submission ID')

,Submission ID,Why do you drink coffee?
3336,0E11K6,It tastes good
10476,0E11K6,I need the ritual
3342,0E1D5Z,It tastes good
14052,0E1D5Z,It makes me go to the bathroom
10482,0E1D5Z,I need the ritual
...,...,...
2918,zKWxaE,It tastes good
10651,zKdykZ,I need the ritual
7081,zKdykZ,I need the caffeine
14221,zKdykZ,It makes me go to the bathroom


In [3]:
age_category_order = [
    '<18 years old',
    '18-24 years old',
    '25-34 years old',
    '35-44 years old',
    '45-54 years old',
    '55-64 years old',
    '>65 years old'
]

most_paid_for_coffee_order = [
    'Less than $2',
    '$2-$4',
    '$4-$6',
    '$6-$8',
    '$8-$10',
    '$10-$15',
    '$15-$20',
    'More than $20'
]

spent_coffee_eqipment_order = [
    'Less than $20',
    '$20-$50',
    '$50-$100',
    '$100-$300',
    '$300-$500',
    '$500-$1000',
    'More than $1,000'
]

# Specify an ordering of the age category from youngest to oldest and convert into categorical data type.

coffee_df['What is your age?'] = pd.Categorical(coffee_df['What is your age?'], categories=age_category_order, ordered=True)
coffee_df["What is the most you've ever paid for a cup of coffee?"] = pd.Categorical(coffee_df["What is the most you've ever paid for a cup of coffee?"], categories=most_paid_for_coffee_order, ordered=True)
coffee_df["Approximately how much have you spent on coffee equipment in the past 5 years?"] = pd.Categorical(coffee_df["Approximately how much have you spent on coffee equipment in the past 5 years?"], categories=spent_coffee_eqipment_order, ordered=True)

In [4]:
coffee_subset = coffee_df[['Submission ID',
           'What is your age?',
            'Gender',
            'Education Level',
            'Employment Status',
           'Do you like the taste of coffee?',
           'Do you know where your coffee comes from?',
            "What is the most you've ever paid for a cup of coffee?",
            'Do you feel like you’re getting good value for your money when you buy coffee at a cafe?',
            'Approximately how much have you spent on coffee equipment in the past 5 years?',
            'Do you feel like you’re getting good value for your money with regards to your coffee equipment?',
            "What is the most you'd ever be willing to pay for a cup of coffee?",
           ]].dropna().reset_index()

Use one-hot encoding for gender, education level, employment status and political affiliation.
Use label encoding for ordinal data- What is your age?, What is the most you've ever paid for a cup of coffee?, Approximately how much have you spent on coffee equipment in the past 5 years?.

But before applying encoding, its' better practice to reduce the number of columns (remove redundant/irrelevent information) to improve the model's performance. Go through a process of feature selection- this will decrease computational costs, reduce overfitting and enhance model interpretability.

As I am working with categorical data, I should use chi-square method.

In [5]:
coffee_subset['What is your age?'].reset_index().sort_values('What is your age?')

,index,What is your age?
2136,2136,<18 years old
295,295,<18 years old
2566,2566,<18 years old
2192,2192,<18 years old
2143,2143,<18 years old
...,...,...
3175,3175,>65 years old
2916,2916,>65 years old
1122,1122,>65 years old
2781,2781,>65 years old


In [17]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder

# x is all the other columns used for prediction and y is your target variable
X = coffee_subset.drop("What is the most you'd ever be willing to pay for a cup of coffee?", axis=1)
y = coffee_subset["What is the most you'd ever be willing to pay for a cup of coffee?"]

# Gather all categorical data (non-ordinal) columns

ordinal_cols = [
    'What is your age?',
    "What is the most you've ever paid for a cup of coffee?",
    'Approximately how much have you spent on coffee equipment in the past 5 years?'
]

non_ordinal_cols = [
    'Gender',
    'Education Level',
    'Employment Status',
    'Do you like the taste of coffee?',
    'Do you know where your coffee comes from?',
    'Do you feel like you’re getting good value for your money when you buy coffee at a cafe?',
    'Do you feel like you’re getting good value for your money with regards to your coffee equipment?'
]

# Apply one-hot encoding to the non-ordinal columns.
non_ordinal_encoded = pd.get_dummies(coffee_subset[non_ordinal_cols])

# Apply label encoding for ordinal data.

# Make a dictionary with the column name as the key and a list with all the values in order.
orders = {
    'What is your age?': [
    '<18 years old',
    '18-24 years old',
    '25-34 years old',
    '35-44 years old',
    '45-54 years old',
    '55-64 years old',
    '>65 years old'
],

    "What is the most you've ever paid for a cup of coffee?": [
    'Less than $2',
    '$2-$4',
    '$4-$6',
    '$6-$8',
    '$8-$10',
    '$10-$15',
    '$15-$20',
    'More than $20'
],

    'Approximately how much have you spent on coffee equipment in the past 5 years?': [
    'Less than $20',
    '$20-$50',
    '$50-$100',
    '$100-$300',
    '$300-$500',
    '$500-$1000',
    'More than $1,000'
]
}

# Create a function that takes in the dataframe, the name of the column and the ordering of the values in each column.
def label_encoding(df,col,order):
    # It creates a mapping with the categoical value (e.g. <18, 18-24, 25-50...) and maps it to an encoding with the order specified (0,1,2).
    mapping = {category: idx for idx, category in enumerate(order)}
    # The mapping is then applied to produce a single series (ordinal_encoded) which the function returns.
    ordinal_encoded = df[col].map(mapping)
    return ordinal_encoded

# Create an empty dataframe to store all the encoded ordinal data.
ordinal_encoded = pd.DataFrame()

# Loop through each key (column title) in the orders dictionary and apply the label encoding function.
# encoded_dfs will be a list of series.
encoded_dfs = [label_encoding(coffee_subset, col, order) for col, order in orders.items()]

# The list of series all are concatenated together into a single dataframe.
ordinal_encoded = pd.concat(encoded_dfs, axis=1)

X_encoded = pd.concat([non_ordinal_encoded, ordinal_encoded], axis=1)
X_encoded

# Now that all the X columns (columns used in the model) have been encoded, we can also encode the target column (Y).
# "What is the most you'd ever be willing to pay for a cup of coffee?"

willing_to_pay_order = [
    'Less than $2',
    '$2-$4',
    '$4-$6',
    '$6-$8',
    '$8-$10',
    '$10-$15',
    '$15-$20',
    'More than $20'
]

Y_encoded = label_encoding(coffee_subset,"What is the most you'd ever be willing to pay for a cup of coffee?",willing_to_pay_order)
Y_encoded

# # Fit a RandomForest model
model = RandomForestClassifier()
model.fit(X_encoded, Y_encoded)

# # Get feature importances
# Feature Importances in a Random Forest model represent the relative importance of each feature in the model's decision-making process.
importances = model.feature_importances_

# This shows the importance of each feature.
feature_importances_df = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Get a list of columns that have a importance of >0.1- these are the features that have more 
# influence on the target the most.
top_features = feature_importances_df[feature_importances_df['Importance']>0.1]['Feature'].to_list()
top_features

# X_encoded
# # Select features based on importance

["What is the most you've ever paid for a cup of coffee?",
 'Approximately how much have you spent on coffee equipment in the past 5 years?',
 'What is your age?']

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# X is the features we are using to make the predictions.
# Y is the target (what we are trying to predict).
X = X_encoded[top_features]
Y = Y_encoded

# We use a part of the dataset to train the model and another part to test how accurate the model is.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Make the random forest classifier model and fit the training data to it.
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
# The accuracy is measured as the number of correct predictions over the total number of predictions.
print(f'Accuracy: {accuracy:.2f}')

print(' ')

print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.38
 
Classification Report:
               precision    recall  f1-score   support

      $10-$15       0.31      0.25      0.28       253
      $15-$20       0.30      0.15      0.20       131
        $2-$4       0.50      0.10      0.17        29
        $4-$6       0.42      0.43      0.43       128
        $6-$8       0.36      0.31      0.33       241
       $8-$10       0.36      0.54      0.43       323
 Less than $2       0.00      0.00      0.00         2
More than $20       0.46      0.50      0.48       219

     accuracy                           0.38      1326
    macro avg       0.34      0.28      0.29      1326
 weighted avg       0.37      0.38      0.36      1326

Confusion Matrix:
[[ 62  19   0   1  22 106   0  43]
 [ 30  19   1   3   4  26   0  48]
 [  1   1   3  15   3   4   1   1]
 [  8   1   1  55  30  29   0   4]
 [  9   4   0  36  74 102   0  16]
 [ 54   2   0  16  61 176   0  14]
 [  0   0   0   1   1   0   0   0]
 [ 33  17   1   3  11  45   0 109]

In [8]:
importances

array([0.01756692, 0.01902405, 0.00792897, 0.00138032, 0.00290009,
       0.03605159, 0.02297962, 0.01372991, 0.00449742, 0.03141206,
       0.0267751 , 0.02686282, 0.01627086, 0.00812861, 0.00659598,
       0.01516837, 0.01243331, 0.00704466, 0.00671515, 0.01197905,
       0.01273187, 0.02232176, 0.02205802, 0.01230465, 0.0128338 ,
       0.15656123, 0.26795432, 0.19778949])

In [9]:
coffee_subset["What is the most you'd ever be willing to pay for a cup of coffee?"].unique()

array(['$8-$10', 'More than $20', '$15-$20', '$4-$6', '$6-$8', '$10-$15',
       '$2-$4', 'Less than $2'], dtype=object)

In [10]:
coffee_subset

,index,Submission ID,What is your age?,Gender,Education Level,Employment Status,Do you like the taste of coffee?,Do you know where your coffee comes from?,What is the most you've ever paid for a cup of coffee?,Do you feel like you’re getting good value for your money when you buy coffee at a cafe?,Approximately how much have you spent on coffee equipment in the past 5 years?,Do you feel like you’re getting good value for your money with regards to your coffee equipment?,What is the most you'd ever be willing to pay for a cup of coffee?
0,34,ylqbBg,45-54 years old,Male,Master's degree,Employed full-time,Yes,Yes,$4-$6,No,$500-$1000,Yes,$8-$10
1,41,YZzBdN,25-34 years old,Male,Master's degree,Unemployed,Yes,Yes,$2-$4,Yes,$50-$100,Yes,More than $20
2,42,VZ17Z6,35-44 years old,Male,Bachelor's degree,Employed full-time,Yes,Yes,$10-$15,Yes,$100-$300,Yes,More than $20
3,43,RzBeZj,45-54 years old,Male,Bachelor's degree,Employed full-time,Yes,Yes,$10-$15,Yes,$300-$500,Yes,$15-$20
4,46,WApbkQ,35-44 years old,Male,Less than high school,Unemployed,Yes,Yes,$6-$8,Yes,"More than $1,000",Yes,$4-$6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3309,4037,PA44VP,>65 years old,Female,Master's degree,Retired,Yes,No,$6-$8,No,Less than $20,Yes,$4-$6
3310,4038,vNgpPD,>65 years old,Male,Bachelor's degree,Retired,Yes,No,$4-$6,No,Less than $20,Yes,$2-$4
3311,4039,g5ggRM,18-24 years old,Male,Some college or associate's degree,Employed full-time,Yes,Yes,$8-$10,Yes,$300-$500,Yes,More than $20
3312,4040,rlgbDN,25-34 years old,Male,Bachelor's degree,Unemployed,Yes,Yes,$4-$6,Yes,$100-$300,Yes,$8-$10


In [11]:
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
mapping

NameError: name 'label_encoder' is not defined

In [ ]:
coffee_df.columns.to_list()

['Submission ID',
 'What is your age?',
 'How many cups of coffee do you typically drink per day?',
 'Where do you typically drink coffee?',
 'Where do you typically drink coffee? (At home)',
 'Where do you typically drink coffee? (At the office)',
 'Where do you typically drink coffee? (On the go)',
 'Where do you typically drink coffee? (At a cafe)',
 'Where do you typically drink coffee? (None of these)',
 'How do you brew coffee at home?',
 'How do you brew coffee at home? (Pour over)',
 'How do you brew coffee at home? (French press)',
 'How do you brew coffee at home? (Espresso)',
 'How do you brew coffee at home? (Coffee brewing machine (e.g. Mr. Coffee))',
 'How do you brew coffee at home? (Pod/capsule machine (e.g. Keurig/Nespresso))',
 'How do you brew coffee at home? (Instant coffee)',
 'How do you brew coffee at home? (Bean-to-cup machine)',
 'How do you brew coffee at home? (Cold brew)',
 'How do you brew coffee at home? (Coffee extract (e.g. Cometeer))',
 'How do you brew

Predicting how much someone would be willing to spend on coffee isn't a very useful model, the only features that actually help with this are how much they've spent on coffee in the past and the most they've spent on coffee in the past. The highest accuracy I could obtain was 0.38.

I will instead try to predict which coffee A, B, C users based on how they answer questions:

How do you brew coffee at home?
On the go, where do you typically purchase coffee?
Do you usually add anything to your coffee?
What kind of dairy do you add?
What kind of sugar or sweetener do you add?
What kind of flavorings do you add?